<a href="https://colab.research.google.com/github/nahbos/AUT-Language-Understanding/blob/main/Ex02/bert_slot_intent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sobhan Moradian Daghigh
### **Ex 02: Slot Filling & Intent detection**

In [1]:
# Copyright @Steven Golovkine: available at [https://stevengolovkine.netlify.app/post/joint-intent-classification-slot-filling-with-transformers/]
# Replicated by @SobhanMoradianDaghigh on 12-8-2022

In [2]:
!pip install transformers==2.11.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 674 kB 19.5 MB/s 
     |████████████████████████████████| 880 kB 50.5 MB/s 
     |████████████████████████████████| 7.5 MB 25.1 MB/s 
     |████████████████████████████████| 1.3 MB 63.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=cc35babc6688f1d327a6362ec219134a297d0babb3a4c86b3414844bdada81d1
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [123]:
# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from pathlib import Path
from transformers import BertTokenizer, TFBertModel
from urllib.request import urlretrieve

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam

In [4]:
!wget -nc https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Ex02/dataset/development-en.conllu
!wget -nc https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Ex02/dataset/test-en.conllu
!wget -nc https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Ex02/dataset/train-en.conllu

--2022-12-09 13:22:53--  https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Ex02/dataset/development-en.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1492856 (1.4M) [text/plain]
Saving to: ‘development-en.conllu’

development-en.conl 100%[===================>]   1.42M  --.-KB/s    in 0.007s  

2022-12-09 13:22:54 (218 MB/s) - ‘development-en.conllu’ saved [1492856/1492856]

--2022-12-09 13:22:54--  https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Ex02/dataset/test-en.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP requ

## Preparing data

In [5]:
def data_preparing(dataset_path):
    prepared = []
    intents = []
    slots = []
    data_list = set(Path(dataset_path).read_text('utf-8').strip().split('\n\n'))
    for data in data_list:
        data_dic = ''
        for line in data.split('\n')[3:]:
            bio = line.split('\t')[-1]    # BIO
            bio = bio if bio != 'NoLabel' else 'O'
            tokens = line.split('\t')[1]  # tokens
            intent = data.split('\n')[1].split(':')[1].strip()  # intent
            data_dic += (tokens + ":" + bio + ' ')
            intents.append(intent)
            slots.append(bio)
        data_dic += '<=> ' + intent
        prepared.append(data_dic)
    return prepared, intents, slots

In [6]:
train, intent_names_train, slots_train = data_preparing('./train-en.conllu')
val, intent_names_val, slots_val       = data_preparing('./development-en.conllu')
test, intent_names_test, slots_test    = data_preparing('./test-en.conllu')

In [7]:
train[:10]

['delete:O a:O reminder:B-reminder/noun <=> reminder/cancel_reminder',
 'is:O it:O going:O to:O rain:B-weather/attribute on:B-datetime tuesday:I-datetime <=> weather/find',
 'is:O there:O rain:B-weather/attribute in:O the:O forecast:B-weather/noun this:B-datetime week:I-datetime <=> weather/find',
 'set:O the:O alarm:O for:B-datetime am:I-datetime every:I-datetime day:I-datetime until:I-datetime friday:I-datetime <=> alarm/set_alarm',
 'what:O will:O the:O high:O temperature:B-weather/noun be:O ?:O <=> weather/find',
 'set:O alarm:O every:B-datetime fifteen:I-datetime minutes:I-datetime <=> alarm/set_alarm',
 'is:O there:O a:O chance:O for:O storms:B-weather/attribute <=> weather/find',
 'repeat:O my:O alarm:O weekly:B-datetime <=> alarm/modify_alarm',
 'set:O alarm:O for:B-datetime 9am:I-datetime on:I-datetime weekends:I-datetime <=> alarm/set_alarm',
 'whats:O weather:B-weather/noun like:O in:O texas:B-location <=> weather/find']

In [8]:
intent_names = list(set(intent_names_train + intent_names_val + intent_names_test))
slots = list(set(slots_train + slots_val + slots_test))

In [9]:
def parse_line(line):
    utterance_data, intent_label = line.split(" <=> ")
    items = utterance_data.split()
    words = [item.rsplit(':', 1)[0] for item in items]
    word_labels = [item.rsplit(':', 1)[1] for item in items]
    return {
        'intent_label': intent_label,
        'words': " ".join(words),
        'words_label': " ".join(word_labels),
        'length': len(words)
    }

In [10]:
parse_line(train[0])

{'intent_label': 'reminder/cancel_reminder',
 'words': 'delete a reminder',
 'words_label': 'O O B-reminder/noun',
 'length': 3}

In [11]:
df_train = pd.DataFrame([parse_line(line) for line in train])
df_validation = pd.DataFrame([parse_line(line) for line in val])
df_test = pd.DataFrame([parse_line(line) for line in test])

In [12]:
df_train.head()

,intent_label,words,words_label,length
0,reminder/cancel_reminder,delete a reminder,O O B-reminder/noun,3
1,weather/find,is it going to rain on tuesday,O O O O B-weather/attribute B-datetime I-datetime,7
2,weather/find,is there rain in the forecast this week,O O B-weather/attribute O O B-weather/noun B-d...,8
3,alarm/set_alarm,set the alarm for am every day until friday,O O O B-datetime I-datetime I-datetime I-datet...,9
4,weather/find,what will the high temperature be ?,O O O O B-weather/noun O O,7


## Intent classification (sentence level)

In [13]:
model_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [14]:
tokenizer.vocab_size

28996

In [15]:
train_sequence_max_length = max([len(tokenizer.encode(text)) for text in df_train['words']])
train_sequence_max_length

34

In [16]:
def encode_dataset(tokenizer, text_sequences, max_length):
    token_ids = np.zeros(shape=(len(text_sequences), max_length), dtype=np.int32)
    for i, text_sequence in enumerate(text_sequences):
        encoded = tokenizer.encode(text_sequence)
        token_ids[i, 0:len(encoded)] = encoded
    attention_masks = (token_ids != 0).astype(np.int32)
    
    return {'input_ids': token_ids, 'attention_masks': attention_masks}

In [17]:
encoded_train      = encode_dataset(tokenizer, df_train['words'], train_sequence_max_length)
encoded_validation = encode_dataset(tokenizer, df_validation['words'], train_sequence_max_length)
encoded_test       = encode_dataset(tokenizer, df_test['words'], train_sequence_max_length)

In [18]:
intent_map = dict((label, idx) for idx, label in enumerate(intent_names))

In [19]:
intent_map

{'reminder/set_reminder': 0,
 'weather/checkSunset': 1,
 'alarm/set_alarm': 2,
 'reminder/cancel_reminder': 3,
 'alarm/snooze_alarm': 4,
 'weather/find': 5,
 'weather/checkSunrise': 6,
 'alarm/modify_alarm': 7,
 'alarm/cancel_alarm': 8,
 'reminder/show_reminders': 9,
 'alarm/show_alarms': 10,
 'alarm/time_left_on_alarm': 11}

In [20]:
intent_train = df_train['intent_label'].map(intent_map).values
intent_validation = df_validation['intent_label'].map(intent_map).values
intent_test = df_test['intent_label'].map(intent_map).values

**Loading and feeding a pretrained BERT model**

In [ ]:
base_bert_model = TFBertModel.from_pretrained('bert-base-cased')
base_bert_model.summary()

In [ ]:
class IntentClassificationModel(tf.keras.Model):
    def __init__(self, intent_num_labels=None, model_name='bert-base-cased', dropout_prob=0.1):
        super().__init__(name='joint_intent_slot')
        # Let's preload the pretrained model BERT in the constructor of our classifier model.
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        
        # Define a (Dense) classification layer to compute for each
        # sequence in a batch of samples. The number of output classes
        # is given by the intent_num_labels parameter.
        # Use the default linear activation (no softmax) to compute
        # logits. The softmax normalization will be computed in the
        # loss function instead of the model itself.
        self.intent_classifier = Dense(intent_num_labels)
        
    def call(self, inputs, **kwargs):
        # Use the pretrained model to extract features from our encoded inputs.
        sequence_output, pooled_output = self.bert(inputs, **kwargs)
        
        # The second output of the main BERT layer has shape:
        # (batch_size, output_dim) and gives a "pooled" representation
        # for the full sequence from the hidden state that corresponds
        # to the "[CLS]" token.
        pooled_output = self.dropout(pooled_output, training=kwargs.get('training', False))
        
        # Use the classifier layer to compute the logits from the pooled features.
        intent_logits = self.intent_classifier(pooled_output)
        return intent_logits

In [ ]:
# Build the model
intent_model = IntentClassificationModel(intent_num_labels=len(intent_map))

intent_model.compile(optimizer=Adam(learning_rate=3e-5, epsilon=1e-08),
                     loss=SparseCategoricalCrossentropy(from_logits=True),
                     metrics=[SparseCategoricalAccuracy('accuracy')])

In [ ]:
# Train the model
history = intent_model.fit(encoded_train, intent_train,
                           epochs=2, batch_size=32,
                           validation_data=(encoded_validation, intent_validation))

In [ ]:
def classify(text, tokenizerzer, model, intent_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :] # Batch size = 1
    class_id = model(inputs).numpy().argmax(axis=1)[0]
    return intent_names[class_id]

In [ ]:
classify('Wake me up for the first job meeting?', tokenizer, intent_model, intent_names)

**WOW))**

## Adding Slot filling (word level) to Intent clf model

In [21]:
slot_names = ["[PAD]"]
slot_names += list(slots)
slot_map = {}
for label in slot_names:
    slot_map[label] = len(slot_map)

In [22]:
slot_map

{'[PAD]': 0,
 'B-weather/noun': 1,
 'I-weather/noun': 2,
 'I-datetime': 3,
 'B-alarm/alarm_modifier': 4,
 'B-datetime': 5,
 'B-reminder/recurring_period': 6,
 'B-timer/noun': 7,
 'B-reminder/todo': 8,
 'I-alarm/alarm_modifier': 9,
 'B-news/type': 10,
 'I-location': 11,
 'B-reminder/noun': 12,
 'B-location': 13,
 'I-reminder/reminder_modifier': 14,
 'I-reminder/recurring_period': 15,
 'B-weather/attribute': 16,
 'B-weather/temperatureUnit': 17,
 'B-demonstrative_reference': 18,
 'O': 19,
 'B-reminder/reminder_modifier': 20,
 'I-demonstrative_reference': 21,
 'B-timer/attributes': 22,
 'B-reminder/reference': 23,
 'I-reminder/reference': 24,
 'B-negation': 25,
 'I-weather/attribute': 26,
 'I-reminder/todo': 27,
 'I-reminder/noun': 28}

In [23]:
def encode_token_labels(text_sequences, slot_names, tokenizer, slot_map, max_length):
    encoded = np.zeros(shape=(len(text_sequences), max_length), dtype=np.int32)
    for i, (text_sequence, word_labels) in enumerate(zip(text_sequences, slot_names)):
        encoded_labels = []
        for word, word_label in zip(text_sequence.split(), word_labels.split()):
            tokens = tokenizer.tokenize(word)
            encoded_labels.append(slot_map[word_label])
            expand_label = word_label.replace("B-", "I-")
            if not expand_label in slot_map:
                expand_label = word_label
            encoded_labels.extend([slot_map[expand_label]] * (len(tokens) - 1))
        encoded[i, 1:len(encoded_labels) + 1] = encoded_labels
    return encoded

In [24]:
slot_train = encode_token_labels(df_train['words'], df_train['words_label'], tokenizer, slot_map, train_sequence_max_length)
slot_validation = encode_token_labels(df_validation['words'], df_validation['words_label'], tokenizer, slot_map, train_sequence_max_length)
slot_test = encode_token_labels(df_test['words'], df_test['words_label'], tokenizer, slot_map, train_sequence_max_length)

In [25]:
class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_num_labels=None, slot_num_labels=None, model_name="bert-base-cased", dropout_prob=0.1):
        super().__init__(name="joint_intent_slot")
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        self.intent_classifier = Dense(intent_num_labels, name="intent_classifier")
        self.slot_classifier = Dense(slot_num_labels, name="slot_classifier")

    def call(self, inputs, **kwargs):
        sequence_output, pooled_output = self.bert(inputs, **kwargs)

        # The first output of the main BERT layer has shape:
        # (batch_size, max_length, output_dim)
        sequence_output = self.dropout(sequence_output, training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(sequence_output)

        # The second output of the main BERT layer has shape:
        # (batch_size, output_dim)
        # and gives a "pooled" representation for the full sequence from the
        # hidden state that corresponds to the "[CLS]" token.
        pooled_output = self.dropout(pooled_output, training=kwargs.get("training", False))
        intent_logits = self.intent_classifier(pooled_output)

        return slot_logits, intent_logits

In [26]:
joint_model = JointIntentAndSlotFillingModel(intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

# Define one classification loss for each output:
opt = Adam(learning_rate=3e-5, epsilon=1e-08)
losses = [SparseCategoricalCrossentropy(from_logits=True), SparseCategoricalCrossentropy(from_logits=True)]
metrics = [SparseCategoricalAccuracy('accuracy')]
joint_model.compile(optimizer=opt, loss=losses, metrics=metrics)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

In [27]:
history = joint_model.fit(
    encoded_train, (slot_train, intent_train),
    validation_data=(encoded_validation, (slot_validation, intent_validation)),
    epochs=2, batch_size=32)

Epoch 1/2
768/768 [==============================] - 215s 256ms/step - loss: 0.2878 - output_1_loss: 0.1046 - output_2_loss: 0.1832 - output_1_accuracy: 0.9727 - output_2_accuracy: 0.9512 - val_loss: 0.0845 - val_output_1_loss: 0.0306 - val_output_2_loss: 0.0539 - val_output_1_accuracy: 0.9920 - val_output_2_accuracy: 0.9843
Epoch 2/2
768/768 [==============================] - 196s 255ms/step - loss: 0.0744 - output_1_loss: 0.0304 - output_2_loss: 0.0441 - output_1_accuracy: 0.9918 - output_2_accuracy: 0.9884 - val_loss: 0.0718 - val_output_1_loss: 0.0242 - val_output_2_loss: 0.0476 - val_output_1_accuracy: 0.9936 - val_output_2_accuracy: 0.9882


In [43]:
def show_predictions(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    slot_logits, intent_logits = outputs
    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    print("## Intent:", intent_names[intent_id])
    print("## Slots:")
    for token, slot_id in zip(tokenizer.tokenize(text), slot_ids):
        print(f"{token:>10} : {slot_names[slot_id]}")

In [44]:
# Example of classification
show_predictions('delete all reminders for this week', tokenizer, joint_model, intent_names, slot_names)

## Intent: reminder/cancel_reminder
## Slots:
       del : O
     ##ete : O
       all : O
  reminder : B-reminder/noun
       ##s : I-reminder/noun
       for : B-datetime
      this : I-datetime
      week : I-datetime


In [97]:
def decode_predictions(text, tokenizer, intent_names, slot_names, intent_id, slot_ids):
    info = {"intent": intent_names[intent_id]}
    collected_slots = {}
    active_slot_words = []
    slot_labels = []
    active_slot_name = None
    for word in text.split():
        tokens = tokenizer.tokenize(word)
        current_word_slot_ids = slot_ids[:len(tokens)]
        slot_ids = slot_ids[len(tokens):]
        current_word_slot_name = slot_names[current_word_slot_ids[0]]
        slot_labels.append(current_word_slot_name)

        if current_word_slot_name == "O":
            if active_slot_name:
                collected_slots[active_slot_name] = " ".join(active_slot_words)
                active_slot_words = []
                active_slot_name = None
        else:
            # Naive BIO: handling: treat B- and I- the same...
            new_slot_name = current_word_slot_name[2:]
            if active_slot_name is None:
                active_slot_words.append(word)
                active_slot_name = new_slot_name
            elif new_slot_name == active_slot_name:
                active_slot_words.append(word)
            else:
                collected_slots[active_slot_name] = " ".join(active_slot_words)
                active_slot_words = [word]
                active_slot_name = new_slot_name
    if active_slot_name:
        collected_slots[active_slot_name] = " ".join(active_slot_words)
    info["slots"] = collected_slots
    return info, slot_labels

In [98]:
def nlu(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    slot_logits, intent_logits = outputs
    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]

    return decode_predictions(text, tokenizer, intent_names, slot_names, intent_id, slot_ids)

In [57]:
output, labels = nlu('delete all reminders for this week', tokenizer, joint_model, intent_names, slot_names)
output

{'intent': 'reminder/cancel_reminder',
 'slots': {'reminder/noun': 'reminders', 'datetime': 'for this week'}}

In [82]:
labels

['O', 'O', 'B-reminder/noun', 'B-datetime', 'I-datetime', 'I-datetime']

In [100]:
def decode_predictions_v2(text, tokenizer, intent_names, slot_names, intent_id, slot_ids):
    slot_labels = []
    for word in text.split():
        tokens = tokenizer.tokenize(word)
        current_word_slot_ids = slot_ids[:len(tokens)]
        slot_ids = slot_ids[len(tokens):]
        current_word_slot_name = slot_names[current_word_slot_ids[0]]
        slot_labels.append(current_word_slot_name)
    return slot_labels

In [126]:
def nlu_v2(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    slot_logits, intent_logits = outputs
    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]

    return decode_predictions_v2(text, tokenizer, intent_names, slot_names, intent_id, slot_ids)

In [129]:
predicted_labels = []
true_labels = []
words_list = []
for data in tqdm(list(df_test.iterrows())):
    intent_label, words, words_label, _ = data[1]
    predicted_label = nlu_v2(words, tokenizer, joint_model, intent_names, slot_names)

    predicted_labels.append(predicted_label)
    true_labels.append(words_label.split(' '))
    words_list.append(words.split(' '))

100%|██████████| 7801/7801 [19:34<00:00,  6.64it/s]


## Evaluation

In [106]:
!wget https://www.comp.nus.edu.sg/%7Ekanmy/courses/practicalNLP_2008/packages/conlleval.pl

--2022-12-09 14:48:07--  https://www.comp.nus.edu.sg/%7Ekanmy/courses/practicalNLP_2008/packages/conlleval.pl
Resolving www.comp.nus.edu.sg (www.comp.nus.edu.sg)... 45.60.35.225
Connecting to www.comp.nus.edu.sg (www.comp.nus.edu.sg)|45.60.35.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12728 (12K)
Saving to: ‘conlleval.pl’

conlleval.pl        100%[===================>]  12.43K  --.-KB/s    in 0.001s  

2022-12-09 14:48:07 (12.0 MB/s) - ‘conlleval.pl’ saved [12728/12728]



In [107]:
import random
import json
import copy
import os
import stat
import subprocess
from os.path import isfile, join
from os import chmod

PREFIX = os.getenv('/your/path/', '') # you should define your perl code directory here


def conlleval(p, g, w, filename):
    '''
    INPUT:
    p :: predictions
    g :: groundtruth
    w :: corresponding words
    OUTPUT:
    filename :: name of the file where the predictions
    are written. it will be the input of conlleval.pl script
    for computing the performance in terms of precision
    recall and f1 score
    '''
    out = ''
    for sl, sp, sw in zip(g, p, w):
        # print(w)
        out += 'BOS O O\n'
        for wl, wp, w in zip(sl, sp, sw):
            out += w + ' ' + wl + ' ' + wp + '\n'
        out += 'EOS O O\n\n'

    f = open(filename, 'w')
    f.writelines(out)
    f.close()

    return get_perf(filename)


def get_perf(filename):
    ''' run conlleval.pl perl script to obtain
    precision/recall and F1 score '''
    _conlleval = PREFIX + './conlleval.pl'
    if not isfile(_conlleval):
        # download('http://www-etud.iro.umontreal.ca/~mesnilgr/atis/conlleval.pl')
        os.system('wget https://www.comp.nus.edu.sg/%7Ekanmy/courses/practicalNLP_2008/packages/conlleval.pl')
        chmod('./conlleval.pl', stat.S_IRWXU)  # give the execute permissions

    proc = subprocess.Popen(["perl", _conlleval], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, _ = proc.communicate(open(filename, 'rb').read())
    for line in stdout.decode("utf-8").split('\n'):
        if 'accuracy' in line:
            out = line.split()
            break

    # out = ['accuracy:', '16.26%;', 'precision:', '0.00%;', 'recall:', '0.00%;', 'FB1:', '0.00']

    precision = float(out[3][:-2])
    recall = float(out[5][:-2])
    f1score = float(out[7])

    return {'p': precision, 'r': recall, 'f1': f1score}


def get_perfo(filename):
    '''
    work around for using a PERL script in python
    dirty but still works.
    '''
    tempfile = str(random.randint(1, np.iinfo('i').max)) + '.txt'
    if not isfile(PREFIX + './conlleval.pl'):
        os.system('wget https://www.comp.nus.edu.sg/%7Ekanmy/courses/practicalNLP_2008/packages/conlleval.pl')
        # download('http://www-etud.iro.umontreal.ca/~mesnilgr/atis/conlleval.pl')
        chmod('conlleval.pl', stat.S_IRWXU)  # give the execute permissions
    if len(PREFIX) > 0:
        chmod(PREFIX + './conlleval.pl', stat.S_IRWXU)  # give the execute permissions
        cmd = PREFIX + './conlleval.pl < %s | grep accuracy > %s' % (filename, tempfile)
    else:
        cmd = './conlleval.pl < %s | grep accuracy > %s' % (filename, tempfile)
    print(cmd)
    out = os.system(cmd)
    out = open(tempfile).readlines()[0].split()
    os.system('rm %s' % tempfile)
    precision = float(out[6][:-2])
    recall = float(out[8][:-2])
    f1score = float(out[10])
    return {'p': precision, 'r': recall, 'f1': f1score}

In [130]:
conlleval(p=predicted_labels, g=true_labels, w=words_list, filename='./results.txt')

{'p': 93.34, 'r': 93.53, 'f1': 93.44}

Finito